<a href="https://colab.research.google.com/github/tvelichkovt/TensorFlow/blob/master/1_TensorFlow_%3E_concepts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test the Environment and TensorFlow ver > 2.x

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
    %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

print('TensorFlow ver is:', tf.__version__, '\nKeras ver is:', keras.__version__, '\nPandas ver is:', pd.__version__, '\nNumpy ver is:', np.__version__)

TensorFlow 2.x selected.
TensorFlow ver is: 2.1.0 
Keras ver is: 2.2.4-tf 
Pandas ver is: 0.25.3 
Numpy ver is: 1.17.5


# def

In [0]:
def f(x):
  return 2 * x + 1

f(5)

11

# tf.nn.relu with linear function


In [0]:
def f(n):
  return x + 1

@tf.function
def deep_net(n):
  return tf.nn.relu(f(n))

deep_net(tf.constant((1, 2, 3)))

NameError: ignored

# ?

In [0]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])
# warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))
print("Note how there's not much difference in performance for convolutions")


# @tf.function

In [0]:
@tf.function
def square_if_positive(x):
  if x > 0:
    x = x * x
  else:
    x = 0
  return x


print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

In [0]:
@tf.function
def sum_even(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s


sum_even(tf.constant([10, 12, 15, 20]))

In [0]:
@tf.function
def fizzbuzz(n):
  for i in tf.range(n):
    if i % 3 == 0:
      tf.print('Fizz')
    elif i % 5 == 0:
      tf.print('Buzz')
    else:
      tf.print(i)

fizzbuzz(tf.constant(15))

In [0]:
class CustomModel(tf.keras.models.Model):

  @tf.function
  def call(self, input_data):
    if tf.reduce_mean(input_data) > 0:
      return input_data
    else:
      return input_data // 2


model = CustomModel()

model(tf.constant([-2, -4]))

In [0]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
  v.assign(v + 1)
  if v % 2 == 0:
    v.assign(v + 1)


find_next_odd()
v

In [0]:
@tf.function
def f(x):
  if x > 0:
    # Try setting a breakpoint here!
    # Example:
    #   import pdb
    #   pdb.set_trace()
    x = x + 1
  return x

tf.config.experimental_run_functions_eagerly(True)

# You can now set breakpoints and run the code in a debugger.
f(tf.constant(1))

tf.config.experimental_run_functions_eagerly(False)

# An in-graph training loop

In [0]:
x = tf.cast(x, tf.float32) / 255.0
y = tf.cast(y, tf.int64)
return x, y

def mnist_dataset():
  (x, y), _ = tf.keras.datasets.mnist.load_data()
  ds = tf.data.Dataset.from_tensor_slices((x, y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.take(20000).shuffle(20000).batch(100)
  return ds

train_dataset = mnist_dataset()

# tf.keras.Sequential

In [0]:
model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))
model.build()
optimizer = tf.keras.optimizers.Adam()

In [0]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


def train_one_step(model, optimizer, x, y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(y, logits)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  compute_accuracy(y, logits)
  return loss


@tf.function
def train(model, optimizer):
  train_ds = mnist_dataset()
  step = 0
  loss = 0.0
  accuracy = 0.0
  for x, y in train_ds:
    step += 1
    loss = train_one_step(model, optimizer, x, y)
    if step % 10 == 0:
      tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
  return step, loss, accuracy

step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())

In [0]:
# Batching

def square_if_positive(x):
  return [i ** 2 if i > 0 else i for i in x] # [-5, -4, -3, -2, -1, 0, 1, 4, 9, 16]

square_if_positive(range(-5, 5))

@tf.function
def square_if_positive_naive(x):
  result = tf.TensorArray(tf.int32, size=x.shape[0])
  for i in tf.range(x.shape[0]):
    if x[i] > 0:
      result = result.write(i, x[i] ** 2)
    else:
      result = result.write(i, x[i])
  return result.stack() #


square_if_positive_naive(tf.range(-5, 5))

In [0]:
# tf.function can give you significant speedup over eager execution, at the cost of a slower first-time execution

import timeit

@tf.function
def f(x, y):
  return tf.matmul(x, y)

print(
    "First invocation:",
    timeit.timeit(lambda: f(tf.ones((10, 10)), tf.ones((10, 10))), number=1))

print(
    "Second invocation:",
    timeit.timeit(lambda: f(tf.ones((10, 10)), tf.ones((10, 10))), number=1))

In [0]:
# @tf.function may also re-trace when called with different non-tensor arguments:

@tf.function
def f(n):
  tf.print(n, '==== test TF ====')

f(9999)


In [0]:
# @tf.function decorator in a nested function:

def f():
  @tf.function
  def f():
    tf.print('==== test TF ====')
  f()

f()

In [0]:
# tf.where 1

def f(x):
  return tf.where(x > 0, x + 99, x)

f(tf.range(-1,1)) # [-1,  0]
f(tf.range(-2,2)) # [ -2,  -1,   0, 100]
f(tf.range(-3,3)) # [ -3,  -2,  -1,   0, 100, 101]
f(tf.range(-4,4)) # [ -4,  -3,  -2,  -1,   0, 100, 101, 102]

In [0]:
# tf.where 2

def f(x):
  return tf.where(x < 0, x + 99, x)

f(tf.range(-4,4)) # [95, 96, 97, 98,  0,  1,  2,  3]
f(tf.range(-2,2)) # [97, 98,  0,  1]

In [0]:
# tf.cast

x = tf.constant([1.8, 2.2], dtype=tf.float32)
tf.dtypes.cast(x, tf.int32)  # [1, 2], dtype=tf.int32


def prepare_mnist_features_and_labels(x, y):
  x = tf.cast(x, tf.float32) / 3.0
  y = tf.cast(y, tf.int32)
  return x, y

prepare_mnist_features_and_labels(2, 5.99) # 0.6666667 float32 & 5 int32

In [0]:
# tf.random.uniform

tf.random.uniform(shape=[4]) # [0.30896592, 0.5679636 , 0.09636641, 0.44236505]
tf.random.uniform(shape=[], minval=-1., maxval=0.) # -0.8583598
tf.random.uniform(shape=[], minval=5, maxval=10, dtype=tf.int64) #8
tf.random.uniform(shape=[], minval=5, maxval=10, dtype=tf.float16) #9.81
tf.random.uniform((3, 3)) # 3*3 array

In [0]:
# tf.matmul 2d + tf.nn.relu => Matrix product of two arrays with Relu -> a transformation that adds non-linearity.

a = tf.constant([1, 2, 3, 4, 5, 6], shape=[2, 3]) # 2-D tensor `a` matrix
b = tf.constant([7, 8, 9, 10, 11, 12], shape=[3, 2]) # 2-D tensor `b` matrix
c = tf.matmul(a, b) # # c1=1*7+2*9+3*11=58 ; c2=1*8+2*10+3*12=64 ; c3=4*7+5*9+6*11=139 ; c4=4*8+5*10+6*12=154 ==> matrix [[ 58,  64],[139, 154]] 
c


@tf.function
def simple_nn_layer(x, y):
  return tf.nn.relu(tf.matmul(x, y))


x = a #or tf.random.uniform((2, 2))
y = b #or tf.random.uniform((2, 2))

simple_nn_layer(x, y)

In [0]:
# tf.matmul 3d with tf.function

a = tf.constant(np.arange(1, 13, dtype=np.int32), shape=[2, 2, 3])
b = tf.constant(np.arange(13, 25, dtype=np.int32), shape=[2, 3, 2])
c = tf.matmul(a, b)
c

@tf.function
def matmul(x, y):
  return tf.matmul(x, y)
x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

matmul(x, y)

In [0]:
# TensorFlow program that uses relu

@tf.function
def relu(vector):
  return tf.nn.relu(vector))


vector = [0., -1.5, 1.5, 2.5, -3.5, -0.1]

relu(vector)

In [0]:
# tf.constant

def linear_layer(x):
  return 2 * x + 1


@tf.function
def deep_net(x):
  return tf.nn.relu(linear_layer(x))


deep_net(tf.constant((1, 2, 3)))

tensor = tf.constant([1, 2, 3, 4, 5, 6, 7]) # vector constant 
tensor # [1, 2, 3, 4, 5, 6, 7]
tensor = tf.constant(-1.0, shape=[2, 3])
tensor # matrix 2/3
tensor = tf.constant((1, 2, 3))
# tensor # [1, 2, 3]

In [7]:
# tf.data.Dataset
#https://www.tensorflow.org/api_docs/python/tf/data/Dataset


dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3]) 
for element in dataset: 
  print(element) 

dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3]) 
dataset = dataset.map(lambda x: x*2) 
list(dataset.as_numpy_iterator()) 





tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


[2, 4, 6]

In [0]:
add1 = tf.add(3, 5)
sess = tf.Session()
print(sess.run(a))
sess.close()

In [0]:
#https://adventuresinmachinelearning.com/python-tensorflow-tutorial/
#https://adventuresinmachinelearning.com/tensorflow-eager-tutorial/

const = tf.constant(2.0, name="const")
b = tf.Variable(2.0, name='b')
c = tf.Variable(1.0, name='c')

b.read_value()
c.read_value()

In [0]:
# now create some operations
d = tf.add(b, c, name='d')
e = tf.add(c, const, name='e')
a = tf.multiply(d, e, name='a')

In [0]:
# setup the variable initialisation
init_op = tf.global_variables_initializer()

In [0]:
# start the session
with tf.Session() as sess:
    # initialise the variables
    sess.run(init_op)
    # compute the output of the graph
    a_out = sess.run(a)
    print("Variable a is {}".format(a_out))

In [0]:
# create TensorFlow variables
b = tf.placeholder(tf.float32, [None, 1], name='b')

In [0]:
a_out = sess.run(a, feed_dict={b: np.arange(0, 10)[:, np.newaxis]})
#https://stackoverflow.com/questions/44151611/how-can-i-solve-this-runtimeerror-attempted-to-use-a-closed-session
#error